# Importing Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd 
import numpy as np
import scipy
import math
from math import sqrt
import os
import tensorflow as ts
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import array 
from collections import Counter
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import math
from collections import Counter
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk.probability import FreqDist
from sklearn.metrics import mean_squared_error

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Downloading SICK dataset

In [ ]:
import requests

def download_sick(f): 

    response = requests.get(f).text

    lines = response.split("\n")[1:]
    lines = [l.split("\t") for l in lines if len(l) > 0]
    lines = [l for l in lines if len(l) == 5]

    df = pd.DataFrame(lines, columns=["idx", "sent_1", "sent_2", "sim", "label"])
    df['sim'] = pd.to_numeric(df['sim'])
    return df
    
sick_train = download_sick("https://raw.githubusercontent.com/alvations/stasis/master/SICK-data/SICK_train.txt")
sick_dev = download_sick("https://raw.githubusercontent.com/alvations/stasis/master/SICK-data/SICK_trial.txt")
sick_test = download_sick("https://raw.githubusercontent.com/alvations/stasis/master/SICK-data/SICK_test_annotated.txt")
sick_all = sick_train.append(sick_test).append(sick_dev)

# Normailizing the "sim" column

In [ ]:
def normalize(df, feature_names):
    result = df.copy()
    for feature_name in feature_names:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [ ]:
sick_all = normalize(sick_all,['sim'])
sick_test = normalize(sick_test,['sim'])
sick_train = normalize(sick_train,['sim'])
sick_dev = normalize(sick_dev,['sim'])

# Preprocessing of lower casing  

In [ ]:
# sick_train['sent_1'] = sick_train['sent_1'].str.lower()
# sick_train['sent_2'] = sick_train['sent_2'].str.lower()

In [ ]:
df1 = sick_train['sent_1'].values
df1

array(['A group of kids is playing in a yard and an old man is standing in the background',
       'A group of children is playing in the house and there is no man standing in the background',
       'The young boys are playing outdoors and the man is smiling nearby',
       ..., 'The man is singing heartily and playing the guitar',
       'A man in blue has a yellow ball in the mitt',
       'Three dogs are resting on a sidewalk'], dtype=object)

In [ ]:
df2 = sick_train['sent_2'].values
len(df2)

4500

In [ ]:
print(len(df1),len(df2))

4500 4500


In [ ]:
import numpy as np
df = np.concatenate((df1,df2), axis=0)
df

array(['A group of kids is playing in a yard and an old man is standing in the background',
       'A group of children is playing in the house and there is no man standing in the background',
       'The young boys are playing outdoors and the man is smiling nearby',
       ...,
       'A bicyclist is holding a bike over his head in a group of people',
       'A man is jumping rope outside',
       'The woman with a knife is slicing a pepper'], dtype=object)

In [ ]:
len(df)

9000

In [ ]:
d_string = ''
for i in range(len(df)):
  s1 = df[i]
  d_string += ' '+(s1) 
#print(d1_string)

In [ ]:
d_string

" A group of kids is playing in a yard and an old man is standing in the background A group of children is playing in the house and there is no man standing in the background The young boys are playing outdoors and the man is smiling nearby The kids are playing outdoors near a man with a smile The young boys are playing outdoors and the man is smiling nearby Two dogs are fighting A brown dog is attacking another animal in front of the man in pants A brown dog is attacking another animal in front of the man in pants Nobody is riding the bicycle on one wheel A person is riding the bicycle on one wheel A person on a black motorbike is doing tricks with a jacket A man with a jersey is dunking the ball at a basketball game A man with a jersey is dunking the ball at a basketball game The player is dunking the basketball into the net and a crowd is in background Two people are kickboxing and spectators are not watching Two young women are sparring in a kickboxing fight Two young women are not

In [ ]:
# def preprocess(text):
     
#       updated = re.sub('<[^<]+?>', '', text) # html tags
#       updated = re.sub(r'\W', ' ', str(text)) # Remove all the special characters
#       updated = re.sub(r'\s+[a-zA-Z]\s+', ' ', text) # remove all single characters
#       updated = re.sub(r'\^[a-zA-Z]\s+', ' ', text) # Remove single characters from the start
#       updated = re.sub(r'\s+', ' ', text, flags=re.I) # substituting multiple spaces with single space
#       updated = re.sub(r'^b\s+', '', text) # removing prefixed b 
#       updated = text.lower()
#       table = str.maketrans({key: None for key in string.punctuation})
#       updated = text.translate(table)
#       return updated

In [ ]:
d_string

" A group of kids is playing in a yard and an old man is standing in the background A group of children is playing in the house and there is no man standing in the background The young boys are playing outdoors and the man is smiling nearby The kids are playing outdoors near a man with a smile The young boys are playing outdoors and the man is smiling nearby Two dogs are fighting A brown dog is attacking another animal in front of the man in pants A brown dog is attacking another animal in front of the man in pants Nobody is riding the bicycle on one wheel A person is riding the bicycle on one wheel A person on a black motorbike is doing tricks with a jacket A man with a jersey is dunking the ball at a basketball game A man with a jersey is dunking the ball at a basketball game The player is dunking the basketball into the net and a crowd is in background Two people are kickboxing and spectators are not watching Two young women are sparring in a kickboxing fight Two young women are not

In [ ]:
#tokens_TF_idf=nltk.wordpunct_tokenize(d_string)

In [ ]:
#len(tokens_TF_idf)

# Preprocessing of text

In [ ]:
# def preprocess(text):
     
#       updated = re.sub('<[^<]+?>', '', text) # html tags
#       updated = re.sub(r'\W', ' ', str(text)) # Remove all the special characters
#       updated = re.sub(r'\s+[a-zA-Z]\s+', ' ', text) # remove all single characters
#       updated = re.sub(r'\^[a-zA-Z]\s+', ' ', text) # Remove single characters from the start
#       updated = re.sub(r'\s+', ' ', text, flags=re.I) # substituting multiple spaces with single space
#       updated = re.sub(r'^b\s+', '', text) # removing prefixed b 
#       table = str.maketrans({key: None for key in string.punctuation})
#       updated = text.translate(table)
#       return updated


In [ ]:
import re

# d_string = [preprocess(sent) for sent in df]

d_string

" A group of kids is playing in a yard and an old man is standing in the background A group of children is playing in the house and there is no man standing in the background The young boys are playing outdoors and the man is smiling nearby The kids are playing outdoors near a man with a smile The young boys are playing outdoors and the man is smiling nearby Two dogs are fighting A brown dog is attacking another animal in front of the man in pants A brown dog is attacking another animal in front of the man in pants Nobody is riding the bicycle on one wheel A person is riding the bicycle on one wheel A person on a black motorbike is doing tricks with a jacket A man with a jersey is dunking the ball at a basketball game A man with a jersey is dunking the ball at a basketball game The player is dunking the basketball into the net and a crowd is in background Two people are kickboxing and spectators are not watching Two young women are sparring in a kickboxing fight Two young women are not

# Lemmatization

In [ ]:
# def lemmatize(d1_tokens):
#     lz = WordNetLemmatizer()
#     lemmatized = [lz.lemmatize(t) for t in d1_tokens]
#     return list(set(lemmatized))

# Stop Words

In [ ]:
en_stop = set(nltk.corpus.stopwords.words('english'))
# from nltk.stem import WordNetLemmatizer
# stemmer = WordNetLemmatizer()

# Tokenization 

In [ ]:
tokens1 = [sent.split() for sent in df1]
for i in range(len(tokens1)):
  # tokens1[i] = [stemmer.lemmatize(word) for word in tokens1[i]]
  #tokens[i] = [word for word in tokens[i] if word not in en_stop]
  tokens1[i] = [word for word in tokens1[i] if len(word) > 2]

In [ ]:
tokens2 = [sent.split() for sent in df2]
for i in range(len(tokens2)):
  # tokens2[i] = [stemmer.lemmatize(word) for word in tokens2[i]]
  #tokens[i] = [word for word in tokens[i] if word not in en_stop]
  tokens2[i] = [word for word in tokens2[i] if len(word) > 2]

In [ ]:
tokens1_with_stop = tokens1
#tokens_without_stop = tokens
# tokens1_with_stop

In [ ]:
tokens1

[['group',
  'kids',
  'playing',
  'yard',
  'and',
  'old',
  'man',
  'standing',
  'the',
  'background'],
 ['group',
  'children',
  'playing',
  'the',
  'house',
  'and',
  'there',
  'man',
  'standing',
  'the',
  'background'],
 ['The',
  'young',
  'boys',
  'are',
  'playing',
  'outdoors',
  'and',
  'the',
  'man',
  'smiling',
  'nearby'],
 ['The', 'kids', 'are', 'playing', 'outdoors', 'near', 'man', 'with', 'smile'],
 ['The',
  'young',
  'boys',
  'are',
  'playing',
  'outdoors',
  'and',
  'the',
  'man',
  'smiling',
  'nearby'],
 ['Two', 'dogs', 'are', 'fighting'],
 ['brown',
  'dog',
  'attacking',
  'another',
  'animal',
  'front',
  'the',
  'man',
  'pants'],
 ['brown',
  'dog',
  'attacking',
  'another',
  'animal',
  'front',
  'the',
  'man',
  'pants'],
 ['Nobody', 'riding', 'the', 'bicycle', 'one', 'wheel'],
 ['person', 'riding', 'the', 'bicycle', 'one', 'wheel'],
 ['person', 'black', 'motorbike', 'doing', 'tricks', 'with', 'jacket'],
 ['man', 'with', 'j

In [ ]:
len(tokens1)

4500

In [ ]:
tokens2_with_stop = tokens2
#tokens_without_stop = tokens
# tokens2_with_stop

In [ ]:
len(tokens2)

4500

In [ ]:
model_tokens = np.concatenate((tokens1,tokens2), axis=0)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
len(model_tokens)

9000

In [ ]:
#d1_lemmas = lemmatize(tokens_with_stop)
#len(d1_lemmas)

In [ ]:
#d2_lemmas = lemmatize(tokens_without_stop)
#len(d2_lemmas)

In [ ]:
STOP = set(nltk.corpus.stopwords.words("english"))

In [ ]:
import nltk
class SentenceModel:

    def __init__(self, sentence):
        self.raw = sentence
        normalized_sentence = sentence.replace("‘", "'").replace("’", "'")
        self.tokens = [t for t in nltk.word_tokenize(normalized_sentence)]
        self.tokens_without_stop = [t for t in self.tokens if t not in STOP]

# Cosine Similarity

In [ ]:
def run_cosine_similarity(sentences1, sentences2, model=None, use_stoplist=False, doc_freqs=None):
    if doc_freqs is not None:
        N = doc_freqs["NUM_DOCS"]

    sims = []
    for (sent1, sent2) in zip(sentences1, sentences2):

        tokens1 = sent1.tokens_without_stop if use_stoplist else sent1.tokens
        tokens2 = sent2.tokens_without_stop if use_stoplist else sent2.tokens
        
        tokens1 = [token for token in tokens1 if token in model]
        tokens2 = [token for token in tokens2 if token in model]

        if len(tokens1) == 0 or len(tokens2) == 0:
            sims.append(0)
            continue

        tokfreqs1 = Counter(tokens1)
        tokfreqs2 = Counter(tokens2)

        weights1 = [tokfreqs1[token] * math.log(N / (doc_freqs.get(token, 0) + 1))
                    for token in tokfreqs1] if doc_freqs else None
        weights2 = [tokfreqs2[token] * math.log(N / (doc_freqs.get(token, 0) + 1))
                    for token in tokfreqs2] if doc_freqs else None

        embedding1 = np.average([model[token] for token in tokfreqs1], axis=0, weights=weights1).reshape(1, -1)
        embedding2 = np.average([model[token] for token in tokfreqs2], axis=0, weights=weights2).reshape(1, -1)

        sim = cosine_similarity(embedding1, embedding2)[0][0]
        sims.append(sim)

    return sims
    

# Word Movers Distance

In [ ]:
from sklearn import preprocessing
def run_wmd_similarity(sentences1, sentences2, model, use_stoplist=False):
    sims = []
    for (sent1, sent2) in zip(sentences1, sentences2):

        tokens1 = sent1.tokens_without_stop if use_stoplist else sent1.tokens
        tokens2 = sent2.tokens_without_stop if use_stoplist else sent2.tokens

        tokens1 = [token for token in tokens1 if token in model]
        tokens2 = [token for token in tokens2 if token in model]

        if len(tokens1) == 0 or len(tokens2) == 0:
            tokens1 = [token for token in sent1.tokens if token in model]
            tokens2 = [token for token in sent2.tokens if token in model]

        sims.append(-model.wmdistance(tokens1, tokens2))
    
    
    x_array = np.array(sims)
    print(x_array)
    normalized_arr = preprocessing.normalize([x_array])
    sims=normalized_arr[0].tolist()
    return sims

    

# Smooth Inverse Frequency

In [ ]:
list1=[]
[list1.append(i) for sent_1 in tokens1_with_stop for i in sent_1]
print(len(list1))

[list1.append(i) for sent_2 in tokens2_with_stop for i in sent_2]
print(len(list1))
frequencies = FreqDist(list1)



29435
58134


In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity


def remove_first_principal_component(X):
    svd = TruncatedSVD(n_components=1, n_iter=7, random_state=0)
    print(X)
    print(type(X))
    svd.fit(X)
    pc = svd.components_
    XX = X - X.dot(pc.transpose()) * pc
    return XX


def run_sif_similarity(sentences1, sentences2, model, freqs={}, use_stoplist=False, a=1):
    total_freq = sum(freqs.values())

    embeddings = []

    # SIF 

    for (sent1, sent2) in zip(sentences1, sentences2):
        tokens1 = sent1.tokens_without_stop if use_stoplist else sent1.tokens
        tokens2 = sent2.tokens_without_stop if use_stoplist else sent2.tokens

        tokens1 = [token for token in tokens1 if token in model]
        tokens2 = [token for token in tokens2 if token in model]

        weights1 = [a / (a + freqs.get(token, 0) / total_freq) for token in tokens1]
        weights2 = [a / (a + freqs.get(token, 0) / total_freq) for token in tokens2]

        embedding1 = np.average([model[token] for token in tokens1], axis=0, weights=weights1)
        embedding2 = np.average([model[token] for token in tokens2], axis=0, weights=weights2)

        embeddings.append(embedding1)
        embeddings.append(embedding2)

    embeddings = remove_first_principal_component(np.array(embeddings))
    sims = [cosine_similarity(embeddings[idx * 2].reshape(1, -1),
                              embeddings[idx * 2 + 1].reshape(1, -1))[0][0]
            for idx in range(int(len(embeddings) / 2))]

    return sims
                

# FastText Model Implementation

In [ ]:
from keras.preprocessing.text import Tokenizer
from gensim.models.fasttext import FastText
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
embedding_size = 600 # 300-dimensional vector
window_size = 2 #the size of the number of words occurring before and after the word based on which the word representations will be learned for the word.
min_word = 1 # the minimum frequency of a word in the corpus for which the word representations will be generated
down_sampling = 1e-2 # the most frequently occurring word will be down-sampled by a number specified by the down_sampling attribute.


In [ ]:
%%time
ft_model = FastText(model_tokens,
                      size=embedding_size,
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                      sg=1,
                      iter=300)

CPU times: user 5min 22s, sys: 549 ms, total: 5min 23s
Wall time: 2min 49s


In [ ]:
print(ft_model.wv['boys'])

[-5.44384599e-01 -1.26114562e-01 -2.51147538e-01  2.43084297e-01
  1.23861402e-01  1.66377604e-01 -5.97226508e-02  2.74319891e-02
 -5.47908768e-02  1.09746888e-01 -2.62255341e-01 -1.00121997e-01
  5.90325482e-02 -5.49711764e-01  6.68524429e-02  1.43693820e-01
  4.10540670e-01  4.23621565e-01 -1.00026153e-01  3.06335419e-01
  2.11731289e-02 -3.19720954e-02 -1.51063696e-01 -8.62728208e-02
  9.28596929e-02 -2.81876802e-01  4.22556810e-02 -5.08631244e-02
  2.24719286e-01  1.46602839e-02 -5.61744422e-02  2.79608369e-01
  2.21424308e-02 -9.88645777e-02  1.66692153e-01 -4.79275793e-01
 -2.71631598e-01  1.01674058e-01  8.50631483e-03  2.27449089e-01
  3.71697545e-01 -7.42091015e-02  4.54205990e-01 -1.49882853e-01
 -2.33546123e-01 -2.22740889e-01  1.45556167e-01  3.49694550e-01
  2.66580969e-01 -3.65778476e-01  2.17321858e-01  3.89185637e-01
 -7.95702860e-02  7.42824981e-03 -2.10218370e-01  1.22102953e-01
 -2.15093315e-01  3.83717082e-02 -2.14385703e-01  1.01512976e-01
  1.35228813e-01 -1.07353

In [ ]:
ft_model.wv.most_similar('kids', topn=10)

[('Kids', 0.6034600734710693),
 ('kid,', 0.5683010220527649),
 ('daschunds', 0.5636042952537537),
 ('boys', 0.5361362099647522),
 ('Boys', 0.5350770354270935),
 ('girls', 0.5287851095199585),
 ('cubs', 0.5055478811264038),
 ('balls', 0.5049699544906616),
 ('Animals', 0.5031906962394714),
 ('games', 0.48711684346199036)]

# Evaluation Metrics

In [ ]:
sentences1 = [SentenceModel(s) for s in sick_train['sent_1']] 
sentences2 = [SentenceModel(s) for s in sick_train['sent_2']]
# sentences1 = [SentenceModel(s) for s in df1] 
# sentences2 = [SentenceModel(s) for s in df2]
#sims = run_cosine_similarity(sentences1,sentences2, model=ft_model, use_stoplist=False) # cosine 
#sims = run_wmd_similarity(sentences1,sentences2, model=ft_model, use_stoplist=False) #word mover distance
sims = run_sif_similarity(sentences1,sentences2, freqs=frequencies, model=ft_model, use_stoplist=False)
#sims = run_context_avg_benchmark(sentences1, sentences2, model=elmo, use_stoplist=False, doc_freqs=True)

sick_train["predicted_sim"] = pd.Series(sims).values
sick_train["diff"] = np.sqrt((sick_train["predicted_sim"] - sick_train["sim"])**2)
# sick_train = sick_train.sort_values("diff", ascending=False)
pearson_correlation = scipy.stats.pearsonr(sims, sick_train['sim'])[0]
spearman_correlation = scipy.stats.spearmanr(sims, sick_train['sim'])[0]
mse = (mean_squared_error(sims, sick_train['sim']))
textstr = 'MSE=%.3f\nPearson Correlation=%.3f\nSpearman Correlation=%.3f'%(mse, pearson_correlation, spearman_correlation)
sick_train['predicted_sim'] = pd.Series(sims).values
sick_train = sick_train.sort_values('sim')
id = list(range(0, len(sick_train.index)))
sick_train['id'] = pd.Series(id).values
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


[[-0.22876924  0.09245212 -0.18834782 ...  0.10030434  0.0303994
  -0.23442866]
 [-0.20672816  0.17942291 -0.11989262 ... -0.08922601  0.03474473
  -0.28033061]
 [-0.15438947  0.13162677 -0.02929515 ... -0.03370781 -0.03637876
  -0.33727424]
 ...
 [-0.3786033   0.30004419 -0.21233192 ...  0.09352877 -0.13904064
  -0.39513118]
 [-0.2647781   0.06642058 -0.07759034 ... -0.1403374  -0.15112982
  -0.28543428]
 [-0.29974362  0.10313816 -0.09970661 ... -0.10785681 -0.13968151
  -0.31419736]]
<class 'numpy.ndarray'>


In [ ]:
print(textstr)

MSE=0.059
Pearson Correlation=0.643
Spearman Correlation=0.568
